In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import pandas as pd

# Dash 애플리케이션 초기화
app = dash.Dash(__name__)

# 애플리케이션 레이아웃 정의
app.layout = html.Div([
    html.Img(src='/Users/admin/softeer/project/assets/팰리세이드.png', style={'width': '100%', 'height': 'auto'}),
    html.H1("팰리세이드 이슈 트래커", style={'textAlign': 'center', 'marginTop': 20, 'marginBottom': 20}),
    
    # 전체 팰리세이드 score 그래프
    html.Div([
        html.H2('Palisade Overall Score', style={'textAlign': 'center', 'marginBottom': 20}),
        dcc.Graph(id='overall-score-graph', style={'width': '100%', 'height': '400px'}),
    ], style={'width': '100%', 'display': 'inline-block', 'marginBottom': 30}),

    # 유사성 대시보드
    html.Div([
        html.H2('Similarity Dashboard', style={'textAlign': 'center', 'marginBottom': 20}),
        html.Div([
            html.Div('기준 차종', style={'marginRight': 10, 'fontSize': 14}),
            dcc.Dropdown(
                id='base-car-dropdown',
                options=[{'label': '팰리세이드', 'value': '팰리세이드'}],
                value='팰리세이드',  # 기본 선택을 팰리세이드로 설정
                style={'width': '90%'}
            )
        ], style={'display': 'flex', 'alignItems': 'center', 'width': '100%', 'marginBottom': 20}),

        html.Div([
            html.Div('유사 Case', style={'marginRight': 10, 'fontSize': 14}),
            dcc.Dropdown(
                id='similarity-dropdown',
                placeholder="Select a similar pattern",
                style={'width': '90%'}
            )
        ], style={'display': 'flex', 'alignItems': 'center', 'width': '100%', 'marginBottom': 20}),

        html.Div([
            html.Div('이슈 날짜', style={'marginRight': 10, 'fontSize': 14}),
            dcc.Dropdown(
                id='issue-date-dropdown',
                placeholder="Select an issue date",
                style={'width': '90%'}
            )
        ], style={'display': 'flex', 'alignItems': 'center', 'width': '100%', 'marginBottom': 20}),

        html.Div(id='selected-info', style={'marginTop': 20, 'fontSize': 18}),
        html.Div(id='issue-details', style={'marginTop': 20, 'fontSize': 18}),
        dcc.Graph(id='similarity-graph', style={'width': '100%', 'height': '400px'}),
    ], style={'width': '100%', 'display': 'inline-block', 'marginBottom': 30}),

    # Sentiment 대시보드와 시간 선택
    html.Div([
        html.H2('Sentiment Dashboard', style={'textAlign': 'center', 'marginBottom': 20}),
        dcc.Graph(id='sentiment-piechart', style={'width': '100%', 'height': '300px'}),
        
        html.Div([
            html.H2('Select Time', style={'textAlign': 'center', 'marginBottom': 20}),
            dcc.Dropdown(
                id='time-slider',
                options=[{'label': f'{i}시', 'value': i} for i in range(24)] + [{'label': 'All', 'value': 24}],
                value=24,
                clearable=False
            ),
            html.Div(id='selected-hour-info', style={'textAlign': 'center', 'marginTop': 20, 'fontSize': 18}),
        ], style={'width': '100%', 'display': 'inline-block', 'marginBottom': 30}),
    ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'}),

    # HOT Issue 대시보드
    html.Div([
        html.H2('HOT Issue Dashboard', style={'textAlign': 'center', 'marginBottom': 20}),
        html.Div(id='hot-issue-info', style={'marginTop': 20, 'fontSize': 18}),
        html.Div(id='hot-issue-title', style={'fontSize': 20, 'marginTop': 20}),
        html.Div(id='hot-issue-context', style={'fontSize': 15, 'marginTop': 10}),
        html.Div(id='hot-issue-comment', style={'fontSize': 15, 'marginTop': 10}),
    ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    
    dcc.Store(id='selected-date'),
    dcc.Store(id='selected-car'),
    dcc.Store(id='selected-similarity-car'),
    dcc.Store(id='selected-datetime'),
])

# 팰리세이드 전체 Score 그래프 업데이트
@app.callback(
    Output('overall-score-graph', 'figure'),
    [Input('base-car-dropdown', 'value')]
)
def update_overall_score_graph(base_car):
    if base_car is None:
        return go.Figure()

    palisade_data = score_df[score_df['CarName'] == base_car]

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=palisade_data['DateTime'],
        y=palisade_data['score'],
        mode='lines',
        name='Palisade Score',
        line=dict(width=2, color='blue')
    ))

    fig.add_shape(
        type='line',
        x0=palisade_data['DateTime'].min(),
        x1=palisade_data['DateTime'].max(),
        y0=70,
        y1=70,
        line=dict(color='red', dash='dash')
    )

    fig.update_layout(
        title='Palisade Overall Score',
        xaxis_title='Date',
        yaxis_title='Score',
        xaxis=dict(range=[palisade_data['DateTime'].min(), palisade_data['DateTime'].max()])
    )

    return fig

# 콜백: 기준 차종 선택 시 유사도 비교 옵션 업데이트
@app.callback(
    Output('similarity-dropdown', 'options'),
    [Input('base-car-dropdown', 'value')]
)
def update_similarity_options(selected_base_car):
    if selected_base_car is None:
        return []

    filtered_df = similarity_df_sorted[similarity_df_sorted['기준차종'] == selected_base_car]
    
    if filtered_df.empty:
        return [{'label': '유사한 차종 없음', 'value': 'None'}]

    options = [
        {'label': f"{row['유사한 차종']} - {row['유사기간']} - 유사도: {row['유사도']:.2f}%", 'value': idx}
        for idx, row in filtered_df.iterrows()
    ]
    return options

# 콜백: 유사 차종 선택 시 이슈 날짜 드롭다운 업데이트
@app.callback(
    Output('issue-date-dropdown', 'options'),
    [Input('similarity-dropdown', 'value')]
)
def update_issue_dates(selected_index):
    if selected_index is None or selected_index == 'None':
        return []

    selected_row = similarity_df_sorted.loc[selected_index]
    similar_car = selected_row['유사한 차종']
    similar_date = pd.to_datetime(selected_row['유사기간'])

    start_date = similar_date - pd.Timedelta(days=14)
    end_date = similar_date + pd.Timedelta(days=14)

    filtered_issues = issue_df[(issue_df['CarName'] == similar_car) &
                               (issue_df['DateTime'] >= start_date) &
                               (issue_df['DateTime'] <= end_date)]
    issue_dates = filtered_issues['DateTime'].dt.strftime('%Y-%m-%d').unique()
    
    issue_dates_sorted = sorted(issue_dates, reverse=True)
    
    return [{'label': date, 'value': date} for date in issue_dates_sorted]

# 콜백: 유사도 대시보드에서 선택한 항목에 대한 그래프 업데이트 및 기준 정보 표시
@app.callback(
    [Output('similarity-graph', 'figure'),
     Output('selected-info', 'children'),
     Output('selected-date', 'data'),
     Output('selected-car', 'data'),
     Output('selected-similarity-car', 'data'),
     Output('selected-datetime', 'data')],
    [Input('similarity-dropdown', 'value'),
     Input('base-car-dropdown', 'value')]
)
def update_similarity_graph_and_slider(selected_index, selected_base_car):
    if selected_index is None or selected_base_car is None:
        return go.Figure(), "No data selected.", None, None, None, None

    try:
        selected_row = similarity_df_sorted.loc[selected_index]
        base_car = selected_row['기준차종']
        base_date = pd.to_datetime(selected_row['기준기간'])
        similar_car = selected_row['유사한 차종']
        similar_date = pd.to_datetime(selected_row['유사기간'])
        similarity_score = selected_row['유사도']

        base_data = score_df[(score_df['CarName'] == base_car) &
                             (score_df['DateTime'] >= base_date - pd.Timedelta(days=14)) &
                             (score_df['DateTime'] < base_date + pd.Timedelta(days=15))]

        similar_data = score_df[(score_df['CarName'] == similar_car) &
                                (score_df['DateTime'] >= similar_date - pd.Timedelta(days=14)) &
                                (score_df['DateTime'] < similar_date + pd.Timedelta(days=15))]

        if base_data.empty or similar_data.empty:
            return go.Figure(), f"No data available for {base_car} or {similar_car} in the selected period.", None, None, None, None

        base_data = base_data.copy()
        similar_data = similar_data.copy()
        base_data['day'] = range(1, len(base_data) + 1)
        similar_data['day'] = range(1, len(similar_data) + 1)

        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x=base_data['day'], 
            y=base_data['score'], 
            mode='lines', 
            name=base_car,
            customdata=base_data['DateTime'],  
            hovertemplate='%{customdata|%Y-%m-%d %H:%M}: %{y}<extra></extra>'
        ))

        fig.add_trace(go.Scatter(
            x=similar_data['day'], 
            y=similar_data['score'], 
            mode='lines', 
            name=similar_car,
            customdata=similar_data['DateTime'],  
            hovertemplate='%{customdata|%Y-%m-%d %H:%M}: %{y}<extra></extra>'
        ))

        issue_events = issue_df[(issue_df['CarName'] == similar_car) &
                                (issue_df['DateTime'] >= similar_date - pd.Timedelta(days=14)) &
                                (issue_df['DateTime'] <= similar_date + pd.Timedelta(days=14))]

        if not issue_events.empty:
            issue_dates = issue_events['DateTime'].dt.strftime('%Y-%m-%d').tolist()
            for issue_date in issue_dates:
                fig.add_vline(x=int((pd.to_datetime(issue_date) - similar_date).days) + 15, 
                              line=dict(color='red', width=2, dash='dash'), 
                              annotation_text=f"Issue: {issue_date}", 
                              annotation_position="top left")

        fig.update_layout(
            title=f'Pattern Similarity | 기준 차종: {base_car} | 유사 차종: {similar_car} | 유사도: {similarity_score:.2f}%',
            xaxis_title='Day',
            yaxis_title='Score',
            xaxis=dict(tickmode='linear', tick0=1, dtick=1, range=[1, 30])
        )

        info_text = f"기준 차종: {base_car} | 기준 날짜: {base_date.date()} | 유사 차종: {similar_car} | 유사도: {similarity_score:.2f}%"
        selected_datetime = pd.to_datetime(f'{base_date.date()} {0:02d}:00:00')

        return fig, info_text, base_date, base_car, similar_car, selected_datetime

    except Exception as e:
        return go.Figure(), f"An error occurred: {str(e)}", None, None, None, None

# Sentiment Piechart 및 시간대별 Sentiment 분석 업데이트 및 HOT Issue 업데이트
@app.callback(
    [Output('sentiment-piechart', 'figure'),
     Output('hot-issue-title', 'children'),
     Output('hot-issue-context', 'children'),
     Output('hot-issue-comment', 'children')],
    [Input('time-slider', 'value'),
     Input('selected-datetime', 'data'),
     Input('selected-car', 'data'),
     Input('selected-similarity-car', 'data'),
     Input('similarity-graph', 'clickData')],
    [State('selected-datetime', 'data')]
)
def update_sentiment_and_hot_issue(selected_hour, selected_datetime, base_car, similar_car, clickData, stored_datetime):
    if clickData:
        selected_datetime = pd.to_datetime(clickData['points'][0]['customdata'])
    elif stored_datetime:
        selected_datetime = pd.to_datetime(stored_datetime)
    else:
        if selected_datetime is None or base_car is None:
            return go.Figure(), "", "", ""

    if selected_hour == 24:  # All을 선택한 경우
        daily_avg_sentiment = sentiment_sum_df[(sentiment_sum_df['Date'] == selected_datetime.date()) & 
                                               ((sentiment_sum_df['CarName'] == base_car) | 
                                                (sentiment_sum_df['CarName'] == similar_car))]

        if daily_avg_sentiment.empty:
            return go.Figure(), "No sentiment data available.", "", ""

        sentiment_data = daily_avg_sentiment.mean(numeric_only=True)  
        selected_datetime = selected_datetime.replace(hour=0)  
    else:
        selected_datetime = selected_datetime.replace(hour=selected_hour)

        sentiment_data = sentiment_sum_df[(sentiment_sum_df['DateTime'] == selected_datetime) & 
                                          ((sentiment_sum_df['CarName'] == base_car) | 
                                           (sentiment_sum_df['CarName'] == similar_car))]

        if sentiment_data.empty:
            return go.Figure(), "No sentiment data available for {}".format(selected_datetime), "", ""

        sentiment_data = sentiment_data.iloc[0]

    sentiment_fig = go.Figure(data=[go.Pie(labels=['긍정', '중립', '부정'],
                                           values=[sentiment_data['긍정'], sentiment_data['중립'], sentiment_data['부정']],
                                           hole=.3)])
    sentiment_fig.update_layout(title=f'Sentiment Analysis for {selected_datetime}')

    post_data = post_df[(post_df['CarName'] == base_car) & 
                        (post_df['DateTime'].dt.date == selected_datetime.date())]

    if selected_hour == 24 and not post_data.empty:
        top_post = post_data.loc[post_data['ViewCount'].idxmax()]
        post_title = f"Title: {top_post['Title']} (All Day)"
        post_content = f"Content: {top_post['Content']}"
    elif not post_data.empty:
        top_post = post_data.loc[post_data['ViewCount'].idxmax()]
        post_title = f"Title: {top_post['Title']}"
        post_content = f"Content: {top_post['Content']}"
    else:
        post_title = "No posts available"
        post_content = "No content available"

    comment_data = comment_df[(comment_df['CarName'] == base_car) & 
                              (comment_df['DateTime'].dt.date == selected_datetime.date())]
    if not comment_data.empty:
        top_comment = comment_data.loc[comment_data['DateTime'].idxmax()]
        comment_content = f"Comment: {top_comment['Content']}"
    else:
        comment_content = "No comments available"

    return sentiment_fig, post_title, post_content, comment_content

if __name__ == '__main__':
    app.run_server(debug=True, port=8050)
